## Implementation of the Paper – EfficientNet-B0 on CIFAR-10

This section documents the baseline implementation of the EfficientNet-B0 model using PyTorch, as described in the original paper [EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks](https://arxiv.org/abs/1905.11946).  
The purpose is to evaluate how well the pretrained model (trained on ImageNet) performs on a new dataset — CIFAR-10 — without any fine-tuning.  
This helps us establish a performance baseline to compare against our later contributions.


### 1. Importing Required Libraries

We begin by importing:
- `torch` and `torch.nn` for deep learning operations
- `torchvision.datasets` and `torchvision.transforms` for loading and preprocessing the CIFAR-10 dataset
- `torchvision.models` for accessing the EfficientNet-B0 architecture with pretrained weights



In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# torchvision provides efficientnet_b0/b1, 
from torchvision.models import efficientnet_b0, efficientnet_b1


### 2. Loading and Preprocessing the CIFAR-10 Dataset

We load the CIFAR-10 dataset using `torchvision.datasets`. It consists of 60,000 color images of size 32×32 pixels, categorized into 10 classes (e.g., airplane, cat, car, etc.). The dataset is split into 50,000 training images and 10,000 test images.

We apply standard preprocessing techniques to prepare the data for training:

- Data Augmentation (Training Set Only):
  - Random cropping with padding
  - Random horizontal flipping
- Normalization:  
  Each image is normalized using the CIFAR-10 dataset's channel-wise mean and standard deviation:
  ```python
  mean = (0.4914, 0.4822, 0.4465)
  std  = (0.2470, 0.2435, 0.2616)


In [2]:

# Prepare the CIFAR-10 dataset and Dataloaders

# Common transforms for CIFAR-10
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),  # mean
                         (0.2470, 0.2435, 0.2616))  # std
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616))
])

# Download/Load CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train
)
test_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test
)

# Create train & test loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)



100%|██████████| 170M/170M [00:02<00:00, 69.0MB/s] 


### 3.Loading the Pretrained Model and Adjusting Output Layer

EfficientNet-B0 is pretrained on ImageNet, which has 1,000 output classes.  
Since CIFAR-10 contains only 10 classes, we modify the final classification layer accordingly.

#### Code:
```python
from torchvision.models import efficientnet_b0
import torch.nn as nn

model = efficientnet_b0(weights='IMAGENET1K_V1')  # Load pretrained model

# Replace the final classifier to output 10 classes instead of 1000
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 10)


In [3]:

# Load Pretrained Model

# Let's pick EfficientNet B0 
model = efficientnet_b0(weights='IMAGENET1K_V1')  # loads pretrained weights

# We need to replace the final classification layer to have 10 outputs for CIFAR-10.
num_ftrs = model.classifier[1].in_features  # typically 1280 for EfficientNet-B0
model.classifier[1] = nn.Linear(num_ftrs, 10)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\selvi/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 52.9MB/s]


In [4]:
# Check the model architecture:
print(model)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

## 4.Evaluating the Pretrained Model (Before Fine-Tuning)

We define an `evaluate()` function to measure the model's test accuracy on the CIFAR-10 dataset.

### The function performs the following steps:
- Sets the model to evaluation mode to disable dropout and batch norm updates.
- Iterates over the test data, passing each batch through the model.
- Computes the predicted class labels by taking the `argmax` of the output logits.
- Compares predictions to the ground truth labels.
- Calculates and returns the overall accuracy.

This evaluation provides a baseline performance of the pretrained EfficientNet-B0 (trained on ImageNet) on CIFAR-10 — before any fine-tuning is applied.

### Purpose:
It helps us understand how well the model's pretrained features generalize to a new, smaller dataset without adaptation.


In [5]:
#  Evaluate performance BEFORE fine-tuning
# we want to see how the pretrained model does on CIFAR-10 as-is, run a quick validation pass:
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100.0 * correct / total
    return accuracy

pre_finetune_acc = evaluate(model, test_loader, device)
print(f"Accuracy of the pretrained (unfine-tuned) model on CIFAR-10: {pre_finetune_acc:.2f}%")


Accuracy of the pretrained (unfine-tuned) model on CIFAR-10: 8.50%


## Notebook Summary

This notebook replicates the original EfficientNet-B0 model using PyTorch, modifies its output layer to match the 10 classes of CIFAR-10, and evaluates its baseline performance without any fine-tuning or training. This establishes a benchmark for comparison before applying transfer learning.
